In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select



In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [4]:
def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  html = browser.page_source
  return html

In [5]:
table_tech = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Emerging_Tech_data_2020\Emerging_tech_master_table_20210929_Emerge_Tech_.csv"
table_tech = pd.read_csv(table_tech)

In [6]:
fields_list = list(set(table_tech['NewSection'].values))

In [7]:
#perform search in Quora and return relevant urls with more than ~100 folloewrs

In [8]:
search_link = 'https://www.quora.com/search?q='

#create search links
all_search_links = []
for field in fields_list:
    field = field.replace(' & ','%20')
    field = field.replace(' ','%20')
    all_search_links.append(search_link+field)


In [9]:
all_search_links

['https://www.quora.com/search?q=Medicine',
 'https://www.quora.com/search?q=Architecture',
 'https://www.quora.com/search?q=IT%20Communication',
 'https://www.quora.com/search?q=Art',
 'https://www.quora.com/search?q=Energy',
 'https://www.quora.com/search?q=Finance',
 'https://www.quora.com/search?q=Construction%20Production',
 'https://www.quora.com/search?q=Military',
 'https://www.quora.com/search?q=Mining',
 'https://www.quora.com/search?q=Legal',
 'https://www.quora.com/search?q=Electronics',
 'https://www.quora.com/search?q=Education',
 'https://www.quora.com/search?q=Space',
 'https://www.quora.com/search?q=Transport',
 'https://www.quora.com/search?q=Human%20Behavior%20Psychology',
 'https://www.quora.com/search?q=Media',
 'https://www.quora.com/search?q=Materials',
 'https://www.quora.com/search?q=Pharma',
 'https://www.quora.com/search?q=Entertainment',
 'https://www.quora.com/search?q=Climate%20Geo',
 'https://www.quora.com/search?q=Agriculture']

In [21]:
class Extract_href_urls():
    def __init__(self, link):
        self.get_group_links(link)
        self.get_topic_urls()
        self.all_hrefs = []
        self.get_hrefs()
          
    def get_group_links(self, link):
        url_html = get_html_from_url(link)
        self.soup = BeautifulSoup(url_html)
        #find all boxes of groups
        all_group_boxes = self.soup.find_all('div', {'class':"q-box qu-pl--medium qu-pr--medium qu-borderBottom qu-tapHighlight--none qu-display--flex qu-alignItems--center"})
        self.groups_names = [box.find_all('span', {'class':"q-text qu-wordBreak--break-word"}) for box in all_group_boxes]
        #keep only text
        self.groups_names = [group[0].find_next(text=True).strip() for group in self.groups_names if group != []]
        self.groups_names = list(set(self.groups_names))

    
    def get_topic_urls(self):
        beg_top_url = 'https://www.quora.com/topic/'
        self.topic_urls = []

        for name in self.groups_names:
            if ' ' in name:
                continue
            topic_url = beg_top_url+name
            #get html to see if there are any stories. If not discard the url
            
            topic_html = get_html_from_url(topic_url)
            soup_topic = BeautifulSoup(topic_html)
            
            stories = soup_topic.find_all('div', {'class':'q-box'})                   
                    
            if stories != []:
                self.topic_urls.append(topic_url)
            

    @staticmethod
    def get_url_tag(box):
        #By find_element not working in selenium using string parsing instead
        url=None
        box = str(box)
        split_box = box.split()
        for word in split_box:
            if 'href' in word:
                url = word.strip('href=').strip('"')
                break
        return url
    
    def get_hrefs(self): 
        for top in self.topic_urls:
            top_html = get_html_from_url(top)
            top_soup = BeautifulSoup(top_html)
       
            all_quests = top_soup.find_all('a', {'class':'q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI'})
            
            for quest in all_quests:
                link_hrefs = self.get_url_tag(quest)
                self.all_hrefs.append(link_hrefs)

             
        
            
        
        

        



In [22]:
res = Extract_href_urls(all_search_links[0])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_53912\3323844770.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached


In [26]:
len(list(set(res.all_hrefs)))



81

In [38]:
ex_link = all_search_links[-1]


In [39]:
ex_link

'https://www.quora.com/search?q=Education'

In [40]:
url_html = get_html_from_url(ex_link)

C:\Users\Ravit\AppData\Local\Temp\ipykernel_48996\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


In [41]:
soup = BeautifulSoup(url_html)

In [42]:
#find all boxes of groups
all_group_boxes = soup.find_all('div', {'class':"q-box qu-pl--medium qu-pr--medium qu-borderBottom qu-tapHighlight--none qu-display--flex qu-alignItems--center"})

In [77]:
all_groups_names = [box.find_all('span', {'class':"q-text qu-wordBreak--break-word"}) for box in all_group_boxes]
#keep only text
all_groups_names = [group[0].find_next(text=True).strip() for group in all_groups_names if group != []]

In [83]:
all_groups_names = list(set(all_groups_names))

In [84]:
all_groups_names


['Educative',
 'Educate!',
 'Quality Skill Consultancy',
 'Educator (company)',
 'Parwarish Cares Foundation',
 'Educative (company)',
 'Pinecrest High School',
 'Shri M.B. Vamdot High School',
 'Educated (book)',
 'Bangladesh Army International University of Science and Technology',
 'EDUCATION',
 'IJLB Publishing',
 'SaiBaba InfoTech',
 'US History',
 'Nehru College of Aeronautics and Applied Sciences',
 'Educators (Indian company)',
 'Educating',
 'Education',
 'Notestore']

In [91]:
beg_top_url = 'https://www.quora.com/topic/'
topic_urls = []

for name in all_groups_names:
    if ' ' in name:
        continue
    topic_url = beg_top_url+name
    topic_urls.append(topic_url)
    

In [92]:
topic_urls

['https://www.quora.com/topic/Educative',
 'https://www.quora.com/topic/Educate!',
 'https://www.quora.com/topic/EDUCATION',
 'https://www.quora.com/topic/Educating',
 'https://www.quora.com/topic/Education',
 'https://www.quora.com/topic/Notestore']

In [93]:

all_a_classes = soup.find_all('a', {'class':"q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--none Link___StyledBox-t2xg9c-0 dxHfBI"})

In [177]:
all_a_classes[1]

<a class="q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--none Link___StyledBox-t2xg9c-0 dxHfBI" href="https://education1.quora.com/" style="box-sizing: border-box; border-radius: inherit;" target="_blank"><div class="q-box qu-pl--medium qu-pr--medium qu-borderBottom qu-hover--bg--darken qu-tapHighlight--none qu-display--flex qu-alignItems--center" style="box-sizing: border-box; position: relative;"><div class="q-flex qu-alignItems--center qu-py--small qu-flex--auto qu-overflow--hidden" style="box-sizing: border-box; display: flex;"><div class="q-box qu-flex--none qu-display--inline-flex qu-mr--small qu-alignSelf--flex-start" style="box-sizing: border-box;"><div class="q-inlineFlex" style="box-sizing: border-box; display: inline-flex;"><div class="q-box qu-display--inline-flex" style="box-sizing: border-box; z-index: 0;"><div class="q-box qu-display--inline-flex" style="box-sizing: border-box;"><div class="q-relative qu-display--inline-flex puppeteer_popper_referen

In [182]:
def get_url_tag(box):
    #By find_element not working in selenium using string parsing instead
    url=None
    box = str(box)
    split_box = box.split()
    for word in split_box:
        if 'href' in word:
            url = word.strip('href=').strip('"')
            break
    return url

In [183]:
print(get_url_tag(all_a_classes[1]))

https://education1.quora.com/


In [175]:
print(get_url_tag(all_a_classes[10]))

https://educations.quora.com/
